In [2]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time

In [3]:
path = '../data/train_set/'
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

In [4]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [5]:
read_time_start=time.time()
X, Y = read_data(path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\liqia\\Spring2021-Project3-group-3-master\\qiaoli/data/train_set//label.csv'

In [7]:
X.shape, Y.shape 

((3000, 3003), (3000,))

In [1]:
print('majority class: %d' % np.sum(Y == 0))
print('minority class: %d' % np.sum(Y == 1))
#imbalanced dataset

NameError: name 'np' is not defined

# oversample then split data

In [8]:
def data_preprocessing(X, Y, path):

  distances = X
  y = Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [9]:
X_balanced, Y_balanced = data_preprocessing(X, Y, path)
X_balanced.shape, Y_balanced.shape

((4804, 3003), (4804,))

In [10]:
print('majority class: %d' % np.sum(Y_balanced == 0))
print('minority class: %d' % np.sum(Y_balanced == 1))
#balanced dataset

majority class: 2402
minority class: 2402


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd

In [12]:
from sklearn.model_selection import train_test_split
X_balanced_train,X_balanced_test,y_balanced_train,y_balanced_test= train_test_split(X_balanced, Y_balanced,test_size=0.2,random_state=0)
X_balanced_train.shape,X_balanced_test.shape,y_balanced_train.shape,y_balanced_test.shape

((3843, 3003), (961, 3003), (3843,), (961,))

In [13]:
print('majority train class: %d' % np.sum(y_balanced_train == 0))
print('minority train class: %d' % np.sum(y_balanced_train == 1))
print('majority test class: %d' % np.sum(y_balanced_test == 0))
print('minority test class: %d' % np.sum(y_balanced_test == 1))

majority train class: 1918
minority train class: 1925
majority test class: 484
minority test class: 477


# split then oversample

In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.20,random_state=0)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print('majority train class: %d' % np.sum(Y_train == 0))
print('minority train class: %d' % np.sum(Y_train == 1))
print('majority test class: %d' % np.sum(Y_test == 0))
print('minority test class: %d' % np.sum(Y_test == 1))
#imbalanced dataset

majority train class: 1923
minority train class: 477
majority test class: 479
minority test class: 121


In [15]:
def data_preprocessing(X_train, Y_train, path):

  distances = X_train
  y = Y_train

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [16]:
X_train_balanced, y_train_balanced = data_preprocessing(X_train, Y_train, path)
X_train_balanced.shape, y_train_balanced.shape

((3846, 3003), (3846,))

In [17]:
def data_preprocessing(X_test, Y_test, path):

  distances = X_test
  y = Y_test

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [48]:
X_test_balanced, y_test_balanced = data_preprocessing(X_test, Y_test, path)
X_test_balanced.shape, y_test_balanced.shape
print('majority train class: %d' % np.sum(y_train_balanced == 0))
print('minority train class: %d' % np.sum(y_train_balanced == 1))
print('majority test class: %d' % np.sum(y_test_balanced == 0))
print('minority test class: %d' % np.sum(y_test_balanced == 1))

majority train class: 1923
minority train class: 1923
majority test class: 479
minority test class: 479


# divide into 2 chunk 3 groups 

        
    • balanced train
    
        1.(train with balanced,test with balanced)
           a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
           b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
        
        2.(train with balanced,test with imbalanced) 
            X_train_balanced.y_train_balanced;X_test,Y_test
      

    • imbalanced train
    
        3. (train with imbalanced,test with imbalanced)
            X_train,Y_train; X_test,Y_test 

# knn

In [19]:
from sklearn.neighbors import NearestCentroid
import numpy as np
from sklearn.metrics import classification_report

In [20]:
# 1.a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
clf = NearestCentroid()
start_time=time.time()
clf.fit(X_balanced_train, y_balanced_train)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 0.079 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.58      0.54      0.56       484
           1       0.56      0.61      0.59       477

    accuracy                           0.57       961
   macro avg       0.57      0.57      0.57       961
weighted avg       0.57      0.57      0.57       961



In [21]:
# 1.b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 0.093 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.49      0.48      0.49       479
           1       0.49      0.51      0.50       479

    accuracy                           0.49       958
   macro avg       0.49      0.49      0.49       958
weighted avg       0.49      0.49      0.49       958



In [49]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.087 seconds
Predicting test data takes 0.01 seconds
              precision    recall  f1-score   support

           0       0.85      0.48      0.61       479
           1       0.24      0.65      0.35       121

    accuracy                           0.51       600
   macro avg       0.54      0.57      0.48       600
weighted avg       0.72      0.51      0.56       600



In [22]:
#3 X_train,Y_train; X_test,Y_test

clf = NearestCentroid()
start_time=time.time()
clf.fit(X_train, Y_train)
NearestCentroid()
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.071 seconds
Predicting test data takes 0.011 seconds
              precision    recall  f1-score   support

           0       0.83      0.53      0.65       479
           1       0.24      0.59      0.34       121

    accuracy                           0.54       600
   macro avg       0.54      0.56      0.49       600
weighted avg       0.71      0.54      0.58       600



In [23]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_balanced_train, y_balanced_train)
    pre=nca_pipe.predict(X_balanced_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_balanced_test, y_balanced_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(y_balanced_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_balanced_train, y_balanced_train)
# Pipeline(...)
pre=nca_pipe.predict(X_balanced_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_balanced_test, y_balanced_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(y_balanced_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_balanced_test)
print(classification_report(y_balanced_test,pre))

the score of a model with k = 3 is 0.703434
the recall of a model with k = 3 is 0.945493
the score of a model with k = 4 is 0.735692
the recall of a model with k = 4 is 0.909853
the score of a model with k = 5 is 0.680541
the recall of a model with k = 5 is 0.970650
the score of a model with k = 6 is 0.715921
the recall of a model with k = 6 is 0.941300
the score of a model with k = 7 is 0.681582
the recall of a model with k = 7 is 0.972746
the score of a model with k = 8 is 0.702393
the recall of a model with k = 8 is 0.953878
the score of a model with k = 9 is 0.665973
the recall of a model with k = 9 is 0.974843
the score of a model with k = 3 is 0.703434
the recall of a model with k = 3 is 0.945493
              precision    recall  f1-score   support

           0       0.90      0.46      0.61       484
           1       0.64      0.95      0.76       477

    accuracy                           0.70       961
   macro avg       0.77      0.71      0.69       961
weighted avg    

In [24]:
# 1.b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train_balanced, y_train_balanced)
    pre=nca_pipe.predict(X_test_balanced)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test_balanced, y_test_balanced)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(y_test_balanced, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train_balanced, y_train_balanced)
# Pipeline(...)
pre=nca_pipe.predict(X_test_balanced)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test_balanced, y_test_balanced)))
print('the recall of a model with k = %d is %f' % (k, recall_score(y_test_balanced, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test_balanced)
print(classification_report(y_test_balanced,pre))

the score of a model with k = 3 is 0.634656
the recall of a model with k = 3 is 0.853862
the score of a model with k = 4 is 0.654489
the recall of a model with k = 4 is 0.795407
the score of a model with k = 5 is 0.607516
the recall of a model with k = 5 is 0.889353
the score of a model with k = 6 is 0.638831
the recall of a model with k = 6 is 0.849687
the score of a model with k = 7 is 0.603340
the recall of a model with k = 7 is 0.903967
the score of a model with k = 8 is 0.618998
the recall of a model with k = 8 is 0.868476
the score of a model with k = 9 is 0.597077
the recall of a model with k = 9 is 0.912317
the score of a model with k = 3 is 0.634656
the recall of a model with k = 3 is 0.853862
              precision    recall  f1-score   support

           0       0.74      0.42      0.53       479
           1       0.59      0.85      0.70       479

    accuracy                           0.63       958
   macro avg       0.67      0.63      0.62       958
weighted avg    

In [50]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train_balanced, y_train_balanced)
    pre=nca_pipe.predict(X_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train_balanced, y_train_balanced)
# Pipeline(...)
pre=nca_pipe.predict(X_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test)
print(classification_report(Y_test,pre))

the score of a model with k = 3 is 0.478333
the recall of a model with k = 3 is 0.727273
the score of a model with k = 4 is 0.535000
the recall of a model with k = 4 is 0.619835
the score of a model with k = 5 is 0.415000
the recall of a model with k = 5 is 0.768595
the score of a model with k = 6 is 0.480000
the recall of a model with k = 6 is 0.685950
the score of a model with k = 7 is 0.396667
the recall of a model with k = 7 is 0.768595
the score of a model with k = 8 is 0.436667
the recall of a model with k = 8 is 0.702479
the score of a model with k = 9 is 0.388333
the recall of a model with k = 9 is 0.809917
the score of a model with k = 3 is 0.478333
the recall of a model with k = 3 is 0.727273
              precision    recall  f1-score   support

           0       0.86      0.42      0.56       479
           1       0.24      0.73      0.36       121

    accuracy                           0.48       600
   macro avg       0.55      0.57      0.46       600
weighted avg    

In [25]:
#3 X_train,Y_train; X_test,Y_test
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline

for k in range(3,10):
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=k)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train, Y_train)
    pre=nca_pipe.predict(X_test)
    # Pipeline(...)
    print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
    print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))
    
    

from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
k=3
nca = NeighborhoodComponentsAnalysis(random_state=42)
knn = KNeighborsClassifier(n_neighbors=k)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train, Y_train)
# Pipeline(...)
pre=nca_pipe.predict(X_test)
# Pipeline(...)
print('the score of a model with k = %d is %f' % (k, nca_pipe.score(X_test, Y_test)))
print('the recall of a model with k = %d is %f' % (k, recall_score(Y_test, pre)))


from sklearn.metrics import classification_report

pre=nca_pipe.predict(X_test)
print(classification_report(Y_test,pre))

the score of a model with k = 3 is 0.776667
the recall of a model with k = 3 is 0.223140
the score of a model with k = 4 is 0.800000
the recall of a model with k = 4 is 0.090909
the score of a model with k = 5 is 0.786667
the recall of a model with k = 5 is 0.148760
the score of a model with k = 6 is 0.800000
the recall of a model with k = 6 is 0.082645
the score of a model with k = 7 is 0.790000
the recall of a model with k = 7 is 0.115702
the score of a model with k = 8 is 0.810000
the recall of a model with k = 8 is 0.082645
the score of a model with k = 9 is 0.803333
the recall of a model with k = 9 is 0.132231
the score of a model with k = 3 is 0.776667
the recall of a model with k = 3 is 0.223140
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       479
           1       0.40      0.22      0.29       121

    accuracy                           0.78       600
   macro avg       0.61      0.57      0.58       600
weighted avg    

# Stochastic Gradient Descent

In [26]:
# 1.a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
# SGD with penalty=l1
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 11.161 seconds
Predicting test data takes 0.011 seconds
              precision    recall  f1-score   support

           0       0.78      0.77      0.78       484
           1       0.77      0.78      0.78       477

    accuracy                           0.78       961
   macro avg       0.78      0.78      0.78       961
weighted avg       0.78      0.78      0.78       961



In [27]:
# 1.b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

# SGD with penalty=l1
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 14.668 seconds
Predicting test data takes 0.008 seconds
              precision    recall  f1-score   support

           0       0.58      0.68      0.63       479
           1       0.62      0.51      0.56       479

    accuracy                           0.59       958
   macro avg       0.60      0.59      0.59       958
weighted avg       0.60      0.59      0.59       958



In [51]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test

# SGD with penalty=l1
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 14.668 seconds
Predicting test data takes 0.089 seconds
              precision    recall  f1-score   support

           0       0.52      0.94      0.67       479
           1       0.68      0.13      0.22       479

    accuracy                           0.53       958
   macro avg       0.60      0.53      0.44       958
weighted avg       0.60      0.53      0.44       958



In [28]:
#3 X_train,Y_train; X_test,Y_test

# SGD with penalty=l1
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l1", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 5.325 seconds
Predicting test data takes 0.005 seconds
              precision    recall  f1-score   support

           0       0.95      0.42      0.58       479
           1       0.28      0.91      0.43       121

    accuracy                           0.52       600
   macro avg       0.62      0.66      0.51       600
weighted avg       0.81      0.52      0.55       600



In [29]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
# SGD with penalty=12
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 5.304 seconds
Predicting test data takes 0.007 seconds
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       484
           1       1.00      0.00      0.00       477

    accuracy                           0.50       961
   macro avg       0.75      0.50      0.34       961
weighted avg       0.75      0.50      0.34       961



C:\Users\liqia\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


In [30]:
# 1.b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
# SGD with penalty=12
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 4.624 seconds
Predicting test data takes 0.01 seconds
              precision    recall  f1-score   support

           0       0.52      0.94      0.67       479
           1       0.68      0.12      0.20       479

    accuracy                           0.53       958
   macro avg       0.60      0.53      0.44       958
weighted avg       0.60      0.53      0.44       958



In [52]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
# SGD with penalty=12
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 4.285 seconds
Predicting test data takes 0.0 seconds
              precision    recall  f1-score   support

           0       1.00      0.02      0.04       479
           1       0.20      1.00      0.34       121

    accuracy                           0.22       600
   macro avg       0.60      0.51      0.19       600
weighted avg       0.84      0.22      0.10       600



In [31]:
#3 X_train,Y_train; X_test,Y_test

# SGD with penalty=12
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="log", penalty="l2", max_iter=200, shuffle=True, class_weight='balanced')
start_time=time.time()
clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 1.543 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       479
           1       0.42      0.49      0.45       121

    accuracy                           0.76       600
   macro avg       0.64      0.66      0.65       600
weighted avg       0.78      0.76      0.77       600



# Neural Networks

In [32]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5,2),
              learning_rate='constant', learning_rate_init=0.001,
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=2,
              shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
start_time=time.time()
clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 0.489 seconds
Predicting test data takes 0.01 seconds
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       484
           1       0.50      1.00      0.66       477

    accuracy                           0.50       961
   macro avg       0.25      0.50      0.33       961
weighted avg       0.25      0.50      0.33       961



C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5,2),
              learning_rate='constant', learning_rate_init=0.001,
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=2,
              shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 0.423 seconds
Predicting test data takes 0.01 seconds
              precision    recall  f1-score   support

           0       0.50      1.00      0.67       479
           1       0.00      0.00      0.00       479

    accuracy                           0.50       958
   macro avg       0.25      0.50      0.33       958
weighted avg       0.25      0.50      0.33       958



C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5,2),
              learning_rate='constant', learning_rate_init=0.001,
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=2,
              shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
start_time=time.time()
clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.521 seconds
Predicting test data takes 0.013 seconds
              precision    recall  f1-score   support

           0       0.80      1.00      0.89       479
           1       0.00      0.00      0.00       121

    accuracy                           0.80       600
   macro avg       0.40      0.50      0.44       600
weighted avg       0.64      0.80      0.71       600



C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5,2),
              learning_rate='constant', learning_rate_init=0.001,
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=2,
              shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
start_time=time.time()
clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 0.384 seconds
Predicting test data takes 0.01 seconds
              precision    recall  f1-score   support

           0       0.80      1.00      0.89       479
           1       0.00      0.00      0.00       121

    accuracy                           0.80       600
   macro avg       0.40      0.50      0.44       600
weighted avg       0.64      0.80      0.71       600



C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\liqia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Decision Tree

In [35]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 5.108 seconds
Predicting test data takes 0.011 seconds
              precision    recall  f1-score   support

           0       0.73      0.47      0.57       484
           1       0.61      0.82      0.70       477

    accuracy                           0.65       961
   macro avg       0.67      0.65      0.63       961
weighted avg       0.67      0.65      0.63       961



In [36]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 5.134 seconds
Predicting test data takes 0.008 seconds
              precision    recall  f1-score   support

           0       0.72      0.27      0.39       479
           1       0.55      0.90      0.68       479

    accuracy                           0.58       958
   macro avg       0.64      0.58      0.54       958
weighted avg       0.64      0.58      0.54       958



In [54]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 5.831 seconds
Predicting test data takes 0.011 seconds
              precision    recall  f1-score   support

           0       0.91      0.27      0.42       479
           1       0.24      0.90      0.38       121

    accuracy                           0.40       600
   macro avg       0.58      0.59      0.40       600
weighted avg       0.78      0.40      0.41       600



In [37]:
#3 X_train,Y_train; X_test,Y_test

from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 2.674 seconds
Predicting test data takes 0.01 seconds
              precision    recall  f1-score   support

           0       0.86      0.71      0.77       479
           1       0.31      0.53      0.39       121

    accuracy                           0.67       600
   macro avg       0.58      0.62      0.58       600
weighted avg       0.75      0.67      0.70       600



# Random Forest & Adaboost

In [38]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 91.754 seconds
Predicting test data takes 0.201 seconds
              precision    recall  f1-score   support

           0       0.85      0.86      0.86       484
           1       0.86      0.85      0.85       477

    accuracy                           0.86       961
   macro avg       0.86      0.86      0.86       961
weighted avg       0.86      0.86      0.86       961



In [39]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 98.876 seconds
Predicting test data takes 0.31 seconds
              precision    recall  f1-score   support

           0       0.72      0.80      0.76       479
           1       0.78      0.69      0.73       479

    accuracy                           0.75       958
   macro avg       0.75      0.75      0.74       958
weighted avg       0.75      0.75      0.74       958



In [55]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 94.417 seconds
Predicting test data takes 0.131 seconds
              precision    recall  f1-score   support

           0       0.83      0.81      0.82       479
           1       0.32      0.36      0.33       121

    accuracy                           0.71       600
   macro avg       0.57      0.58      0.58       600
weighted avg       0.73      0.71      0.72       600



In [40]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 33.408 seconds
Predicting test data takes 0.165 seconds
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       479
           1       0.68      0.12      0.21       121

    accuracy                           0.81       600
   macro avg       0.75      0.55      0.55       600
weighted avg       0.79      0.81      0.76       600



In [41]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 398.47 seconds
Predicting test data takes 401.48 seconds
              precision    recall  f1-score   support

           0       0.86      0.81      0.83       484
           1       0.82      0.87      0.84       477

    accuracy                           0.84       961
   macro avg       0.84      0.84      0.84       961
weighted avg       0.84      0.84      0.84       961



In [42]:
#1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 432.111 seconds
Predicting test data takes 836.371 seconds
              precision    recall  f1-score   support

           0       0.78      0.73      0.75       479
           1       0.75      0.79      0.77       479

    accuracy                           0.76       958
   macro avg       0.76      0.76      0.76       958
weighted avg       0.76      0.76      0.76       958



In [56]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 386.623 seconds
Predicting test data takes 388.423 seconds
              precision    recall  f1-score   support

           0       0.84      0.73      0.78       479
           1       0.30      0.46      0.37       121

    accuracy                           0.68       600
   macro avg       0.57      0.60      0.57       600
weighted avg       0.73      0.68      0.70       600



In [43]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 256.084 seconds
Predicting test data takes 1094.426 seconds
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       479
           1       0.53      0.42      0.47       121

    accuracy                           0.81       600
   macro avg       0.69      0.66      0.67       600
weighted avg       0.79      0.81      0.80       600



In [44]:
#1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 224.617 seconds
Predicting test data takes 1320.294 seconds
              precision    recall  f1-score   support

           0       0.84      0.80      0.82       484
           1       0.81      0.84      0.83       477

    accuracy                           0.82       961
   macro avg       0.82      0.82      0.82       961
weighted avg       0.82      0.82      0.82       961



In [45]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 227.602 seconds
Predicting test data takes 1549.553 seconds
              precision    recall  f1-score   support

           0       0.75      0.72      0.73       479
           1       0.73      0.76      0.74       479

    accuracy                           0.74       958
   macro avg       0.74      0.74      0.74       958
weighted avg       0.74      0.74      0.74       958



In [57]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 192.366 seconds
Predicting test data takes 581.598 seconds
              precision    recall  f1-score   support

           0       0.84      0.72      0.77       479
           1       0.28      0.44      0.34       121

    accuracy                           0.66       600
   macro avg       0.56      0.58      0.56       600
weighted avg       0.72      0.66      0.69       600



In [46]:
#3 X_train,Y_train; X_test,Y_test

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=100)
start_time=time.time()
clf = clf.fit(X_train, Y_train)
#scores = cross_val_score(clf, img_set, label, cv=5)
#print('the 5-fold cross validation score for AdaBoost with 100 estimators is %f' % scores.mean())
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start_time=time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 123.66 seconds
Predicting test data takes 1674.172 seconds
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       479
           1       0.49      0.32      0.39       121

    accuracy                           0.80       600
   macro avg       0.67      0.62      0.63       600
weighted avg       0.77      0.80      0.78       600

